In [1]:
from aicsimageio import AICSImage
import os

import numpy as np
import napari_segment_blobs_and_things_with_membranes as nsbatwm
import napari_process_points_and_surfaces as nppas
import vedo
from skimage import measure

import napari
import napari_stress

ModuleNotFoundError: No module named 'aicsimageio'

In [ ]:
# Config
surface_density = 0.1  # vertices/microns^2

In [ ]:
filename = r'C:\Users\johamuel\Desktop\3-001.czi'
image = AICSImage(filename)

Since we can not move all data to the memory, we shall process one frame at a time. In this notebook, we'll focus on demonstrating the workflow *for a single timepoint*. Still large enough:

In [ ]:
image.dask_data[0]

In [ ]:
scale = np.asarray(image.physical_pixel_sizes)
print('Voxel sizes: ', scale)

In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(np.asarray(image.dask_data[0, 0]), name = 'Spheroid', colormap='bop orange',
                 scale=scale, blending='additive',
                contrast_limits=[154.0, 4410.0])

napari.utils.nbscreenshot(viewer, canvas_only=True)

## Droplet analysis

before we look at the spheroid, let's have a look at the droplet. Due to the large array size, reshaping the data is not an option, so we'll have to go straight to points and surfaces from here.

### Preprocessing

Preprocessing follows a slightly different scheme here. The steps include:

- Thresholding (simple threshold otsu)
- Surface reconstruction (marching cubes)
- Surface decimation (We don't need all the >100k vertices of the surface do achieve a good representation)

Before we proceed, it makes sense to crop a part of the data which we can reshape to heterotropic voxel size

In [ ]:
# Binarize and crop the droplet based on its bounding box
droplet_binary = nsbatwm.threshold_otsu(image.dask_data[0, 1])
props = measure.regionprops_table(droplet_binary, intensity_image=image.dask_data[0, 1], properties=['image', 'bbox', 'image_intensity'])
droplet_cropped_bin = props['image'][0]
droplet_cropped_int = props['image_intensity'][0]

Since we crop out a part of the image, we need to make sure that we are putting the results at the correct location

In [ ]:
translate = np.asarray([props[f'bbox-{i}'] for i in range(3)], dtype=int).flatten() * scale

In [ ]:
dropplet_resampled_int = napari_stress.resample(droplet_cropped_int, vsz=scale[0], vsy=scale[1], vsx=scale[2])

viewer.add_labels(droplet_cropped_bin, translate=translate, scale=scale, name='Droplet cropped')
viewer.add_image(dropplet_resampled_int, translate=translate, scale=[np.min(scale)]*3, blending='additive', colormap='cyan')

Now we can retrieve the surface from this data:

In [ ]:
surface = list(nppas.label_to_surface(droplet_cropped_bin))
surface[0] = surface[0] * scale

In [ ]:
# Adjust vertex density: Distribute N points on surface and recreate surface
mesh = vedo.mesh.Mesh((surface[0], surface[1]))
vertices = nppas.sample_points_uniformly(surface, number_of_points=int(mesh.area() * surface_density))
surf = napari_stress.reconstruct_surface(vertices)

In [ ]:
# Adjust vertex density: Decimate to desired number of vertices
mesh = vedo.mesh.Mesh((surf[0], surf[1]))
mesh.decimate(N=int(mesh.area() * surface_density))
surf = (mesh.points(), np.asarray(mesh.faces()))
viewer.add_surface(surf, translate=translate, name='density-adjusted surface')

### Surface refinement

With the decimated surface, we can now apply the developed surface tracing method in `napari-stress`. Caveat: Is this legitimate to do for non-isotropic image data??

In [ ]:
result = napari_stress.trace_refinement_of_surface(dropplet_resampled_int,
                                                   (surf[0], surf[1]),
                                                   trace_length=30.0, sampling_distance=1,
                                                   show_progress=True,
                                                   selected_fit_type='quick',
                                                   scale=scale,
                                                   remove_outliers=True,
                                                   interquartile_factor=3.0)

In [ ]:
pts_layer = viewer.add_points(np.stack(result.surface_points.to_numpy()), size=1, face_color='orange', translate=translate)

In [ ]:
pts_layer.world_to_data?